In [7]:
import matplotlib.pyplot as plt
import numpy as np
from   numpy.linalg import inv
import numpy.random as npr
from   pypolyagamma import PyPolyaGamma
from scipy.stats import multivariate_normal

import matplotlib.cm as cm
from tqdm import tqdm
import polars as pl

import geopandas as gpd

from shapely.geometry import Point

import pandas as pd
import statsmodels.api as sm
import scipy
import scipy.optimize as opt
import seaborn as sns

from bayesian_statistics.utils_2 import *
import japanize_matplotlib

from keplergl import KeplerGl

### obsedian_gdfの読み込み

In [8]:
obsedian_df = pd.read_csv("/home/ohta/dev/bayesian_statistics/obsedian_data/obsedian.csv")
df = obsedian_df.copy()
# 緯度経度が0のデータを削除
df = df[(df['緯度'] != 0) & (df['経度'] != 0)]

# 黒曜石産地のカラムを選択
obsidian_sources = ['高原山', '和田峠', '男女倉', '諏訪', '蓼科', '神津島', '箱根', '天城', 'その他', '不明']

# 各黒曜石産地の合計を計算
df[df['合計'] != df[obsidian_sources].sum(axis=1)]
df['合計'] = df[obsidian_sources].sum(axis=1)
# 遺跡名でグループ分けして、緯度と経度、メッシュコードと都道府県を残して、obsidian_sourcesのカラムたちと「合計」カラムはsumする
grouped_df = df.groupby('遺跡名').agg({
    '緯度': 'first',
    '経度': 'first',
    'メッシュコード': 'first',
    '都道府県': 'first',
    **{source: 'sum' for source in obsidian_sources},
    '合計': 'sum'
}).reset_index()

grouped_df
grouped_df["合計"].sum()
# GeoDataFrameに変換
obsedian_gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.経度, df.緯度), crs="EPSG:4326"
)
obsedian_grouped_gdf = gpd.GeoDataFrame(
    grouped_df, geometry=gpd.points_from_xy(grouped_df.経度, grouped_df.緯度), crs="EPSG:4326"
)

### gdf_elevation

In [9]:
# 1次メッシュ
mesh_codes = [
    '5138', '5139', 
    '5238', '5239', '5240',
    '5338', '5339', '5340',
    '5438', '5439', '5440',
    '5538', '5539', '5540', '5541'
]

gdfs = []
for mesh_code in mesh_codes:
    path_shp = f"../data/elevation_data/G04-d-11_{mesh_code}-jgd_GML"
    gdf = gpd.read_file(path_shp)
    gdfs.append(gdf)

gdf = pd.concat(gdfs, ignore_index=True)

# カラム名の対応辞書を作成
column_mapping = {
    'G04d_001': 'mesh_code_5th',
    'G04d_002': 'average_elevation',
    'G04d_003': 'maximum_elevation',
    'G04d_004': 'minimum_elevation',
    'G04d_005': 'minimum_elevation_code',
    'G04d_006': 'maximum_slope_angle',
    'G04d_007': 'maximum_slope_direction',
    'G04d_008': 'minimum_slope_angle',
    'G04d_009': 'minimum_slope_direction',
    'G04d_010': 'average_slope_angle'
}

column_dtypes = {
    'mesh_code_5th': 'str',
    'average_elevation': 'float',
    'maximum_elevation': 'float',
    'minimum_elevation': 'float',
    'minimum_elevation_code': 'str',
    'maximum_slope_angle': 'float',
    'maximum_slope_direction': 'str',
    'minimum_slope_angle': 'float',
    'minimum_slope_direction': 'str',
    'average_slope_angle': 'float'
}

# GeoDataFrameのカラム名を変更
gdf = gdf.rename(columns=column_mapping)

# "unknown"をNaNに変換
gdf = gdf.replace('unknown', np.nan)

# カラムのデータ型を変換
gdf = gdf.astype(column_dtypes)

gdf_elevation = gdf

In [10]:
gdf_elevation['x'] = gdf_elevation.apply(lambda x: x.geometry.centroid.x, axis=1)
gdf_elevation['y'] = gdf_elevation.apply(lambda x: x.geometry.centroid.y, axis=1)

In [11]:
obsedian_gdf.時期名.unique()

array(['草創期', '草創期後半', '草創期(隆起線文〜爪形文)', '草創期(隆線文)', '草創期〜早期前葉(大谷寺Ⅱ・Ⅲ〜井草)',
       '草創期(隆線文)', '草創期(無文土器)', '早期前葉(稲荷台)', '早期前葉(押型文)', '早期(無紋)',
       '早期前葉(平坂・押型文)', '早期後葉(平坂)', '早期前葉', '早期前葉(撚糸文)', '早期前葉(井草Ⅱ〜稲荷台)',
       '早期中葉〜後葉(三戸〜条痕文)', '早期(撚糸文末〜沈線文)', '早期中葉〜後葉(条痕文)', '早期中葉(田戸上層)',
       '早期(押型文〜田戸下層)', '早期後葉(鵜ヶ島台)', '早期末(下吉井・打越)', '早期末(条痕文)',
       '早期末〜前期初頭', '早期後葉(野島・鵜ヶ島台・茅山下層)', '早期後葉(子母口主体)', '早期後葉(茅山上層)',
       '早期後葉', '早期後葉(野島)', '早期〜後期(野島・茅山上層多い)', '早期後葉〜末葉(神ノ木台)', '早期後葉〜末葉',
       '早期後葉(八ツ崎)', '早期後葉(茅山)', '早期〜後期(条痕文系が6割程度)', '早期後葉(条痕文)',
       '早期末葉(打越期)', '早期末葉', '早期後葉(茅山下層)', '早期後葉(条痕文系/野島・鵜ヶ島台)',
       '早期後葉(条痕文系)', '早期(条痕文系)', '前期前葉(関山Ⅰ)', '前期前葉', '前期前葉(二ツ木)',
       '前期前葉(関山Ⅱ)', '前期前葉(関山)', '前期初頭(花積下層)', '前期(関山〜黒浜)', '前期(関山主体)',
       '前期中葉(有尾)', '前期中葉(黒浜)', '前期中葉(黒浜・有尾)', '前期中葉(黒浜1)', '前期中葉〜後葉',
       '前期中葉〜後葉(黒浜〜諸磯a)', '前期後葉(諸磯b〜c)', '前期後葉(諸磯b)', '前期後葉(諸磯a）',
       '前期後葉(諸磯b古)', '前期後葉(諸磯b新)', '前期後葉(諸磯a)', '前期後葉(浮島・興津)',
       '前期後葉(諸磯a新〜b古)', '前期後葉(諸磯a新〜b

In [12]:
path_lake_shp = "../data/lake_data/W09-05_GML"
gdf_lake = gpd.read_file(path_lake_shp)

lake_column_name = {
    'W09_001': 'lake_name', 
    'W09_002': 'lake_code',
    'W09_003': 'max_depth',
    'W09_004': 'elevation'
}
lake_column_dtypes = {
    'lake_name': 'str',
    'lake_code': 'str',
    'max_depth': 'float',
    'elevation': 'float'
}

gdf_lake = gdf_lake.rename(columns=lake_column_name)
gdf_lake = gdf_lake.astype(lake_column_dtypes)

In [13]:
gdf_lake

,lake_name,lake_code,max_depth,elevation,geometry
0,さっぽろ湖,01100,NaN,NaN,"POLYGON ((141.15681 42.98894, 141.15678 42.989..."
1,定山湖,01100,NaN,NaN,"POLYGON ((141.14571 42.90153, 141.14572 42.901..."
2,オタルナイ湖,01203,NaN,NaN,"POLYGON ((141.04395 43.13354, 141.04398 43.133..."
3,春採湖,01206,NaN,NaN,"POLYGON ((144.41118 42.9769, 144.41117 42.9768..."
4,富里湖,01208,NaN,NaN,"POLYGON ((143.74607 43.86033, 143.74612 43.860..."
...,...,...,...,...,...
551,フンガー湖,47301,NaN,NaN,"POLYGON ((128.27916 26.75389, 128.27912 26.753..."
552,伊集の湖,47301,NaN,NaN,"POLYGON ((128.26398 26.79097, 128.264 26.79104..."
553,福上湖,47303,NaN,NaN,"POLYGON ((128.22718 26.66008, 128.22723 26.660..."
554,新川湖,47303,NaN,NaN,"POLYGON ((128.23847 26.69218, 128.23854 26.691..."


In [14]:
# river_node_dataディレクトリから"Stream"を含むファイルを削除
!find ../data/river_node_data -type f -name "*Stream*" -exec rm {} \;

# river_stream_dataディレクトリから"RiverNode"を含むファイルを削除
!find ../data/river_stream_data -type f -name "*RiverNode*" -exec rm {} \;

In [15]:
# 都道府県コード
pref_codes = [
    "07_07", 
    "08_08", "08_09", "08_10", "08_11", "08_12", "08_13", "08_14", 
    "07_15", "08_19", "08_20", 
    "08_22"
]

river_node_column_name = {
    'W05_001': 'area_code',
    'W05_011': 'elevation',
    'W05_000': 'river_node_id'
}

river_node_column_dtypes = {
    'area_code': 'str',
    'elevation': 'float',
    'river_node_id': 'str'
}
    

# river_node_data
gdfs = []
for pref_code in pref_codes:
    path_shp = f"../data/river_node_data/W05-{pref_code}_GML"
    gdf = gpd.read_file(path_shp)
    gdfs.append(gdf)

gdf_river_node = pd.concat(gdfs, ignore_index=True)

gdf_river_node = gdf_river_node.rename(columns=river_node_column_name)
gdf_river_node = gdf_river_node.astype(river_node_column_dtypes)


In [16]:
# river_stream_data
#
#  W05_001からW05_010まで
river_stream_column_name = {
    'W05_001': 'water_system_code',    # 水系域コード
    'W05_002': 'river_code',           # 河川コード
    'W05_003': 'river_type',           # 区間種別
    'W05_004': 'river_name',           # 河川名
    'W05_005': 'source_material_type', # 原典資料種別
    'W05_006': 'flow_direction',       # 流下方向判定
    'W05_007': 'river_start_node',     # 河川始点
    'W05_008': 'river_end_node',       # 河川終点
    'W05_009': 'route_start_node',     # 流路始点
    'W05_010': 'route_end_node'        # 流路終点
}

river_stream_column_dtypes = {
    'water_system_code': 'str',
    'river_code': 'str',
    'river_type': 'int',
    'river_name': 'str',
    'source_material_type': 'str',
    'flow_direction': 'bool',
    'river_start_node': 'str',
    'river_end_node': 'str',
    'route_start_node': 'str',
    'route_end_node': 'str'
}

gdfs = []
for pref_code in pref_codes:
    path_shp = f"../data/river_stream_data/W05-{pref_code}_GML"
    gdf = gpd.read_file(path_shp)
    gdfs.append(gdf)

gdf_river_stream = pd.concat(gdfs, ignore_index=True)
gdf_river_stream = gdf_river_stream.rename(columns=river_stream_column_name)
gdf_river_stream = gdf_river_stream.astype(river_stream_column_dtypes)


In [17]:
# 指定されたエリアの境界
min_lon, max_lon = 138.0, 141.0
min_lat, max_lat = 34.0, 37.0

# エリア内に入っているかどうかを判定する関数
def filter_gdf_by_area(gdf, min_lon, max_lon, min_lat, max_lat):
    return gdf.cx[min_lon:max_lon, min_lat:max_lat]

# 各gdfデータをフィルタリング
gdf_elevation =        filter_gdf_by_area(gdf_elevation, min_lon, max_lon, min_lat, max_lat)
gdf_lake =             filter_gdf_by_area(gdf_lake, min_lon, max_lon, min_lat, max_lat)
gdf_river_node =       filter_gdf_by_area(gdf_river_node, min_lon, max_lon, min_lat, max_lat)
gdf_river_stream =     filter_gdf_by_area(gdf_river_stream, min_lon, max_lon, min_lat, max_lat)
obsedian_gdf =         filter_gdf_by_area(obsedian_gdf, min_lon, max_lon, min_lat, max_lat)
obsedian_grouped_gdf = filter_gdf_by_area(obsedian_grouped_gdf, min_lon, max_lon, min_lat, max_lat)

In [18]:
gdf_elevation.to_csv("../data/gdf_elevation.csv", index=False)
gdf_lake.to_csv("../data/gdf_lake.csv", index=False)
gdf_river_node.to_csv("../data/gdf_river_node.csv", index=False)
gdf_river_stream.to_csv("../data/gdf_river_stream.csv", index=False)
obsedian_gdf.to_csv("../data/obsedian_gdf.csv", index=False)
obsedian_grouped_gdf.to_csv("../data/obsedian_grouped_gdf.csv", index=False)

In [19]:
import geopandas as gpd
from shapely.geometry import Polygon

def create_bounding_box(min_lon, max_lon, min_lat, max_lat):
    """
    Create a rectangular polygon from given coordinates
    
    Parameters:
    min_lon: Minimum longitude (western boundary)
    max_lon: Maximum longitude (eastern boundary)
    min_lat: Minimum latitude (southern boundary)
    max_lat: Maximum latitude (northern boundary)
    
    Returns:
    GeoDataFrame containing the bounding box polygon
    """
    # Create coordinates for the polygon (counterclockwise)
    coordinates = [
        (min_lon, min_lat),  # Bottom left
        (max_lon, min_lat),  # Bottom right
        (max_lon, max_lat),  # Top right
        (min_lon, max_lat),  # Top left
        (min_lon, min_lat)   # Back to start to close the polygon
    ]
    
    # Create polygon
    polygon = Polygon(coordinates)
    
    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(
        {'name': ['関東地方の範囲']},
        geometry=[polygon],
        crs="EPSG:4326"
    )
    
    return gdf

# 指定された範囲でバウンディングボックスを作成
bounding_box = create_bounding_box(
    min_lon=138.0,
    max_lon=141.0,
    min_lat=34.0,
    max_lat=37.0
)

# bounding_boxをcsv保存
bounding_box.to_csv("../data/bounding_box.csv", index=False)

| 属性名（かっこ内はshp属性名） | 説明 | 属性の型 |
| --- | --- | --- |
| 5次メッシュコード（G04d_001） | 標準地域メッシュの5次メッシュ | 文字列型（CharacterString） |
| 平均標高（G04d_002） | 10mメッシュの標高値から算出する平均標高（m） | 実数型（Real）※整備データが存在しない場合、“unknown”とする。 |
| 最高標高（G04d_003） | 10mメッシュの標高値から算出する最高標高（m） | 実数型（Real）※整備データが存在しない場合、“unknown”とする。 |
| 最低標高（G04d_004） | 10mメッシュの標高値から算出する最低標高（m） | 実数型（Real）※整備データが存在しない場合、“unknown”とする。 |
| 最低標高コード（G04d_005） | 最低標高が海面下であるかどうかの区分。※基盤地図情報の標高では判定できないため一律“0”としている。 | コードリスト型（海面下=5、その他=0） |
| 最大傾斜角度（G04d_006） | 10mメッシュ標高をリサンプリングした50mメッシュ標高から算出する最大傾斜角度（度） | 実数型（Real）※整備データが存在しない場合、“unknown”とする。 |
| 最大傾斜方向（G04d_007） | 10mメッシュ標高をリサンプリングした50mメッシュ標高から算出する最大傾斜の方向 | コードリスト型（0=方向なし、1=北、2=北東、3=東、4=東南、5=南、6=南西、7=西、8=北西）※整備データが存在しない場合、“unknown”とする。 |
| 最小傾斜角度（G04d_008） | 10mメッシュ標高をリサンプリングした50mメッシュ標高から算出する最小傾斜角度（度） | 実数型（Real）※整備データが存在しない場合、“unknown”とする。 |
| 最小傾斜方向（G04d_009） | 10mメッシュ標高をリサンプリングした50mメッシュ標高から算出する最小傾斜の方向 | コードリスト型（0=方向なし、1=北、2=北東、3=東、4=東南、5=南、6=南西、7=西、8=北西）※整備データが存在しない場合、“unknown”とする。 |
| 平均傾斜角度（G04d_010） | 10mメッシュ標高をリサンプリングした50mメッシュ標高から算出する傾斜角度の平均値（度） | 実数型（Real）※整備データが存在しない場合、“unknown”とする。 |